In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
path = "../../../data/cleaned/multi_var_wind_hourly_cleaned.csv"
required_columns = ['datetime', 'pressure', 'temperature', 'humidity', 'wind_speed', 'u', 'v']
dtype_cast = {'wind_speed': float}


In [3]:
import os
import sys

# Get the absolute path of the directory containing the notebook (ensemble)
# Then get the parent directory's parent (which is the project root: wind-forecast-benchmark)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Add the project root to the system path
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [4]:
from modules.preprocessing import *

INFO:modules.preprocessing:preprocessing module loaded (v1.0.0)


In [5]:
pwd

'/home/amir/Desktop/wind-forecast-benchmark/notebooks/non-linear'

In [6]:
df_basic_clean = load_and_basic_clean(path=path,
                                      required_columns=required_columns,
                                      sort_by='datetime')

INFO:modules.preprocessing:Loaded CSV from '../../../data/cleaned/multi_var_wind_hourly_cleaned.csv': 10329 rows, 10 columns
INFO:modules.preprocessing:Selected 7 required columns
INFO:modules.preprocessing:Sorted DataFrame by column 'datetime'
INFO:modules.preprocessing:Data loading and cleaning completed successfully


In [7]:
df_basic_clean = df_basic_clean[0]

In [8]:
df_time_features = add_time_features(df_basic_clean)

In [9]:
df_time_features.head()

,datetime,pressure,temperature,humidity,wind_speed,u,v,hour,month,day_of_week,hour_sin,hour_cos
0,2024-01-01 00:00:00,758.466667,-1.016667,62.166667,0.100000,-0.022693,-0.047340,0,1,0,0.000000,1.000000
1,2024-01-01 01:00:00,758.383333,-1.533333,64.000000,0.283333,-0.113825,0.011167,1,1,0,0.258819,0.965926
2,2024-01-01 02:00:00,758.383333,-1.150000,61.333333,0.200000,-0.169286,-0.061781,2,1,0,0.500000,0.866025
3,2024-01-01 03:00:00,758.783333,-1.166667,58.666667,0.500000,0.142045,-0.397690,3,1,0,0.707107,0.707107
4,2024-01-01 04:00:00,759.000000,-1.483333,62.166667,0.966667,-0.512590,-0.609574,4,1,0,0.866025,0.500000


In [10]:
df_time_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     10329 non-null  datetime64[ns]
 1   pressure     10329 non-null  float64       
 2   temperature  10329 non-null  float64       
 3   humidity     10329 non-null  float64       
 4   wind_speed   10329 non-null  float64       
 5   u            10329 non-null  float64       
 6   v            10329 non-null  float64       
 7   hour         10329 non-null  int32         
 8   month        10329 non-null  int32         
 9   day_of_week  10329 non-null  int32         
 10  hour_sin     10329 non-null  float64       
 11  hour_cos     10329 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int32(3)
memory usage: 847.4 KB


In [11]:
physical_featurs = ['temperature', 'pressure', 'u', 'v', 'wind_speed']

In [12]:
df_features = add_lag_and_rolling_features(df_time_features,target_cols=physical_featurs, lags=[1, 2, 3 ,6, 12 ,24], dropna=True)

In [13]:
df_features.head(3)

,datetime,pressure,temperature,humidity,wind_speed,u,v,hour,month,day_of_week,...,wind_speed_roll_min_3,wind_speed_roll_max_3,wind_speed_roll_mean_6,wind_speed_roll_std_6,wind_speed_roll_min_6,wind_speed_roll_max_6,wind_speed_roll_mean_12,wind_speed_roll_std_12,wind_speed_roll_min_12,wind_speed_roll_max_12
24,2024-01-02 00:00:00,760.400000,-1.933333,89.166667,1.050000,-0.871871,-0.322368,0,1,1,...,0.333333,0.90,0.625000,0.289780,0.333333,1.00,1.093056,1.098195,0.183333,3.316667
25,2024-01-02 01:00:00,760.216667,-2.183333,89.666667,0.933333,-0.889509,-0.096850,1,1,1,...,0.716667,1.05,0.744444,0.293194,0.333333,1.05,0.904167,0.847221,0.183333,2.933333
26,2024-01-02 02:00:00,760.200000,-2.416667,84.666667,0.700000,-0.572663,-0.323016,2,1,1,...,0.716667,1.05,0.822222,0.265344,0.333333,1.05,0.737500,0.559677,0.183333,2.216667


In [14]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10305 entries, 24 to 10328
Columns: 102 entries, datetime to wind_speed_roll_max_12
dtypes: datetime64[ns](1), float64(98), int32(3)
memory usage: 8.0 MB
